**Zadania robione w SWISH prolog (przeglądarka**

**Zadanie 1**

In [ ]:
% faky kobiety
kobieta(gabriela).
kobieta(alina).
kobieta(sylwia).
kobieta(monika).
kobieta(halina).
kobieta(patrycja).
kobieta(izabela).
kobieta(amelia).

% fakty mężczyzni
mezczyzna(jerzy).
mezczyzna(krzysztof).
mezczyzna(daniel).
mezczyzna(witold).
mezczyzna(adam).
mezczyzna(maksym).
mezczyzna(andrzej).
mezczyzna(tomasz).
mezczyzna(igor).
mezczyzna(artur).

% fakty małżeństwa
malzenstwo(jerzy, gabriela).
malzenstwo(monika, daniel).
malzenstwo(witold, halina).
malzenstwo(adam, patrycja).
malzenstwo(andrzej, izabela).
malzenstwo(tomasz, amelia).

% malzenstwo dziala obustronnie
malzenstwo(X, Y) :- malzenstwo(Y, X), !.

% fakty rodzice
ojciec(jerzy, alina).
ojciec(jerzy, krzysztof).
ojciec(daniel, sylwia).
ojciec(witold, daniel).
ojciec(witold, gabriela).
ojciec(adam, maksym).
ojciec(andrzej, jerzy).
ojciec(andrzej, patrycja).
ojciec(andrzej, tomasz).
ojciec(igor, tomasz).
ojciec(igor, artur).

matka(gabriela, alina).
matka(gabriela, krzysztof).
matka(monika, sylwia).
matka(halina, daniel).
matka(halina, gabriela).
matka(patrycja, maksym).
matka(izabela, jerzy).
matka(izabela, patrycja).
matka(izabela, tomasz).
matka(amelia, tomasz).
matka(amelia, artur).

% reguła rodzica: matka lub ojciec.

rodzic(X,Y) :- ojciec(X,Y) ; matka(X,Y).

% reguła rodzenstwa
brat(X,Y) :-
    rodzic(Z, X),
    rodzic(Z, Y),  %ci sami rodzice
    X \= Y,       %X i Y nie są tą samą osobą
    mezczyzna(X).  %X to mezczyzna
siostra(X,Y):-
    rodzic(Z,X),
    rodzic(Z,Y),
    X \= Y,
    kobieta(X). %X to kobieta
rodzenstwo(X,Y):-siostra(X,Y) ; brat(X,Y).

% Reguła dziadka i babci
dziadek(X,Y):-
    rodzic(Z,Y), % Y (wnuczek) jest dzieckiem Z
    ojciec(X,Z). % X (dziadek) jest ojcem Z (rodzicem Y)
babcia(X,Y):-
    rodzic(Z,Y),
    matka(X,Z).

%reguła kuzynowstwa

kuzyn(X, Y) :-
    rodzic(A,X),       %A to rodzic X (A jest rodzicem pierwszego z kuzynółw-X)
    rodzic(B,Y),       %B to rodzic Y (B jest rodzicem drugiego z kuzynów-Y)
    rodzenstwo(A,B),   %A i B są rodzeństwem
    X \= Y.

%reguła przodka (osoba od której ktoś pochodzi, bezpośrednio lub pośrednio, poprzez linię przodków (rodzice, dziadkowie, pradziadkowie itd.)
przodek(X,Y) :-
    (rodzic(X,Y);  %rodzic jest przodkiem swojego dziecka
    rodzic(X,Z), przodek(Z,Y)). %rodzic naszego przodka też jest naszym przodkiem.


In [ ]:
?- przodek(andrzej, jerzy). % Oczekiwane: true, zwrócone: true
?- przodek(jerzy, izabela).  %Oczekiwane: false, zwrócone: false
?- brat(krzysztof, alina). % Oczekiwane: true, zwrócone: false
?- brat(krzysztof, daniel). %Oczekiwane: false, zwrócone: false
?- brat(alina,krzysztof). %Oczekiwane: false, zwrócone: false
?- kuzyn(X,krzysztof). %Oczekiwane: maksym, sylwia,  zwrócone: X=sylwia, X=maksym
?- kuzyn(X,sylwia). %Oczekiwane: alina, krzysztof,  zwrócoe: X=Alina, X=Krzysztof

---
**Zadanie 2:**


In [ ]:
lotnisko(warszawa).
lotnisko(krakow).
lotnisko(gdansk).
lotnisko(wroclaw).
lotnisko(poznan).
lotnisko(katowice).
lotnisko(szczecin).
lotnisko(lodz).
polaczenie(warszawa, krakow, 300).
polaczenie(krakow, gdansk, 200).
polaczenie(warszawa, gdansk, 200).
polaczenie(warszawa, wroclaw, 180).
polaczenie(warszawa, poznan, 170).
polaczenie(krakow, katowice, 100).
polaczenie(gdansk, szczecin, 130).
polaczenie(wroclaw, poznan, 140).
polaczenie(poznan, szczecin, 120).
polaczenie(katowice, lodz, 110).
polaczenie(lodz, wroclaw, 160).

% czy jest bezpośrednie połączenie między
polaczenie(X,Y) :-
    polaczenie(X,Y,_), !;
    polaczenie(Y,X,_),!.

% bezpośrednie połączenie
koszt_podrozy(X,Y,C) :-
    polaczenie(X,Y,C),!; %połączenie od X do Y
    polaczenie(Y,X,C),!.  %połączenie w drugą stronę (od Y do X)

% wyszukiwanie najtańszej trasy poprzez rekurencje
najkrotsza_droga(X,Y,C) :-
    koszt_podrozy(X,Y,C),!.  % bezpośrednie połączenie

% najkrotsza droga, gdy nie ma bezposredniego
najkrotsza_droga(X, Y) :-
    znajdz_droge(X, Y, [X], T, C),  %znajdz trase przez i cene: T-trasa C=koszt
    format("trasa przez ~w, ~w.", [T, C]), !.  % Wyświetl wynik


znajdz_droge(X, X, T, T, 0).  % jesli wpiszemy jako poczatek i koniec to samo miejsce, to koszt=0
znajdz_droge(X, Y, D, P, C) :- %D-dotychczasowa droga, P-pełna droga
    koszt_podrozy(X, Z, C1),  % znajdź połączenie z X do Z
    znajdz_droge(Z, Y, [Z | D], P, C2),  % przez rekurencje znajdź dalszą trasę
    C is C1 + C2,!.  % wynik jest suma poprzednich czesci

In [ ]:
?- polaczenie(warszawa, krakow). %oczekiwane: true, zwrócone: true
?- polaczenie(gdansk, krakow). %oczekiwane: true, zwrócone: true
?- polaczenie(woclaw,gdansk). %oczekiwane: false, zwrócone: false
?- koszt_podrozy(warszawa, gdansk, C). %oczekiwane: C=200, zwrócone: C=200
?- koszt_podrozy(warszawa, krakow, C). %oczekiwane: C=300, zwrócone: C=300
?- najkrotsza_droga(warszawa, gdansk). %oczekiwane: trasa przez:*nazwy miast*, *koszt*, zwrócone:trasa przez[gdansk,krakow,warszawa], 500.
?- najkrotsza_droga(warszawa, szczecin). %oczekiwane: trasa przez:*nazwy miast*, *koszt*, zwrócone: trasa przez [szczecin,gdansk,krakow,warszawa], 630.


---
**Zadanie3**

In [ ]:
:- dynamic produkt/3. %produkt może byc modyfikowany

produkt(tv, 10, sektor_a).
produkt(pralka, 5, sektor_b).
produkt(lodowka, 3, sektor_c).
produkt(mikrofala, 9, sektor_d).
produkt(radio, 7, sektor_e).
produkt(słuchawki, 20, sektor_f).
produkt(czajnik, 15, sektor_g).
produkt(blender, 7, sektor_h).
produkt(zamrazalka, 5, sektor_i).
produkt(szuszarka, 8, sektor_j).
produkt(prostownica, 5, sektor_k).
produkt(gofrownica, 4, sektor_l).
produkt(toster, 25, sektor_m).
produkt(drukarka, 2, sektor_n).
produkt(lampka, 5, sektor_o).
produkt(odkurzacz, 9, sektor_p).
produkt(wiertarka, 0, sektor_r).

%dostepny produkt reguła
dostepny_produkt(X) :-
    produkt(X,Y,_), %X-> nazwa produktu, Y ilość
    Y>0. %Y ilosc produktu jest wieksza niz 0

%przenies produkt
przenies_produkt(X, Y) :-
    produkt(X, L, S),  % X-nazwa, L-liczba, S-stare polozenie
    retract(produkt(X, L, S)),  % usuwa stare dane
    assert(produkt(X, L, Y)),  % dodaje nowe dane
    format("Produkt (~w, ~w, ~w)", [X, L, Y]).

%uzupelnij stan
uzupelnij_stan(X,Y):-
    produkt(X,L,S),  % X-nazwa, L-liczba, S-sektor
    retract(produkt(X, L, S)),  % usuwa stare dane
    N is L + Y, %N-nowa ilosc = L-Liczbie aktualnej + Y-tym, o ile uzupelniamy
    assert(produkt(X, N, S)), %dodaje nowe, podane dane
	format("Produkt (~w, ~w, ~w)", [X,N,S]).

In [ ]:
?- dostepny_produkt(tv). %oczekiwane: true, zwrócone: true
?- dostepny_produkt(wiertarka). %oczekiwane: false, zwrócone: false
?- dostepny_produkt(zmywarka). %oczekiwane: false, zwrócone: false
?- przenies_produkt(tv, sektor_b). %oczekiwane: produkt(tv, 10, sektor_b), zwrócone: Produkt (tv, 10, sektor_b)
?- przenies_produkt(pralka, sektor_z). %oczekiwane: Produkt (pralka, 5, sektor_z), zwrócone: Produkt (pralka, 5, sektor_z)
?- uzupelnij_stan(pralka, 3). %oczekiwane: Produkt(pralka, 8, sektor_b), zwrócone: Produkt(pralka, 8, sektor_b)
?- uzupelnij_stan(wiertarka, 4). %oczekiwane: Produkt (wiertarka, 4, sektor_r), zwrócone: Produkt (wiertarka, 4, sektor_r)
?- uzupelnij_stan(lampka, 10). %oczekiwane: Produkt (lampka, 15, sektor_o), zwrócone: Produkt (lampka, 15, sektor_o)

---
**Zadanie4**

In [ ]:
miasto(warszawa).
miasto(krakow).
miasto(poznan).
miasto(szczecin).
miasto(lodz).
miasto(gdansk).
miasto(lublin).
miasto(radom).
miasto(wroclaw).
miasto(zielonagora).
miasto(gdynia).
miasto(katowice).
droga(warszawa, krakow, 4).
droga(warszawa, poznan, 3.5).
droga(warszawa, szczecin, 6).
droga(warszawa, lodz, 1.5).
droga(warszawa, gdansk, 4).
droga(warszawa, lublin, 2).
droga(warszawa, radom, 1.3).
droga(warszawa, wroclaw, 4).
droga(warszawa, zielonagora, 5).
droga(warszawa, gdynia, 4).
droga(warszawa, katowice, 3.5).
droga(krakow, poznan, 5).
droga(krakow, szczecin, 7).
droga(krakow, lodz, 2.8).
droga(krakow, gdansk, 6.5).
droga(krakow, lublin, 3.5).
droga(krakow, radom, 2.3).
droga(krakow, wroclaw, 3).
droga(krakow, zielonagora, 4.5).
droga(krakow, gdynia, 6.7).
droga(krakow, katowice, 1.1).
droga(poznan, szczecin, 3).
droga(poznan, lodz, 2.5).
droga(poznan, gdansk, 3.5).
droga(poznan, lublin, 5).
droga(poznan, radom, 4).
droga(poznan, wroclaw, 2).
droga(poznan, zielonagora, 1.7).
droga(poznan, gdynia, 3.7).
droga(poznan, katowice, 4).
droga(szczecin, lodz, 5.5).
droga(szczecin, gdansk, 4.5).
droga(szczecin, wroclaw, 4.5).
droga(szczecin, zielonagora, 2.7).
droga(szczecin, gdynia, 4.7).
droga(szczecin, katowice, 7).
droga(lodz, gdansk, 4).
droga(lodz, lublin, 3.8).
droga(lodz, radom, 2).
droga(lodz, wroclaw, 2.7).
droga(lodz, zielonagora, 4.3).
droga(lodz, gdynia, 4.2).
droga(lodz, katowice, 2.5).
droga(gdansk, lublin, 5.5).
droga(gdansk, radom, 4.7).
droga(gdansk, wroclaw, 5.5).
droga(gdansk, zielonagora, 5).
droga(gdansk, gdynia, 0.7).
droga(gdansk, katowice, 6).
droga(lublin, radom, 1.5).
droga(lublin, wroclaw, 5.5).
droga(lublin, gdynia, 5.7).
droga(lublin, katowice, 4).
droga(radom, wroclaw, 4.5).
droga(radom, zielonagora, 5.8).
droga(radom, gdynia, 5).
droga(radom, katowice, 2.8).
droga(wroclaw, zielonagora, 2).
droga(wroclaw, gdynia, 5.5).
droga(wroclaw, katowice, 2.5).
droga(zielonagora, gdynia, 5.3).
droga(zielonagora, katowice, 4.5).

droga(X,Y):-
    droga(X,Y,_),!;
    droga(Y,X,_),!.

% Czas przejazdu między miastami
czas_przejazdu(X, Y, T) :-
    droga(X, Y, T),!; % bezpośrednia droga
    droga(X, Z, T1), % znajdź pośrednie miasto-Z
    czas_przejazdu(Z, Y, T2),  % rekurencyjnie oblicz czas Z -> Y
    T is T1 + T2,!.              % czas przejazdu to suma innych


% najkrotsza droga, gdy nie ma bezposredniego
najkrotsza_trasa(X, Y,T) :-
    znajdz_trase(X, Y, [X], D, T),  %znajdz trase przez i cene: D-droga T-czas
    format("Trasa przez ~w czas", [D]), !.  % Wyświetl wynik


znajdz_trase(X, X, B, B, 0).  % jesli wpiszemy jako poczatek i koniec to samo miejsce, to koszt=0
znajdz_trase(X, Y, D, P, T) :- %D-dotychczasowa droga, P-pełna droga
    czas_przejazdu(X, Z, T1),  % znajdź połączenie z X do Z
    znajdz_trase(Z, Y, [Z | D], P, T2),  % przez rekurencje znajdź dalszą trasę
    T is T1 + T2,!.  % wynik jest suma poprzednich czesci


In [ ]:
?- droga(warszawa, krakow). %oczekiwane: true, zwrócone: true
?- droga(szczecin,lublin).  %oczekiwane: false, zwrócone: false
?- czas_przejazdu(warszawa, poznan, T). %oczekiwane: T=_, zwrócone: T=3.5
?- czas_przejazdu(szczecin,lublin,T). %oczekiwane: T=_, zwrócone: T=9.3
?- najkrotsza_trasa(warszawa, poznan,T). %oczekiwane: trasa przez *miejsca*, czas T=*czas*  zwrócone: Trasa przez [poznan,krakow,warszawa] czas T = 9
?- najkrotsza_trasa(lublin, szczecin, T). %oczekiwane trasa przez *miejsca*, czas T=*czas*  zwrócone: , zwróconeTrasa przez [lublin,gdansk,lodz,szczecin] czas T = 15.0

---
**Zdanie5**

In [ ]:
ksiazka('W pustyni i w puszczy', sienkiewicz, przygodowa).
ksiazka('Hobbit', tolkien, fantasy).
ksiazka('Pan Tadeusz', mickiewicz, poezja).
ksiazka('Lalka', prus, powiesc).
ksiazka('Dziady', mickiewicz, dramat).
ksiazka('Zemsta', fredro, komedia).
ksiazka('Koziołek Matołek', makuszynski, dla_dzieci).
ksiazka('Metamorfoza', kafka, nowela).
ksiazka('Harry Potter i Kamień Filozoficzny', rowling, fantasy).
ksiazka('Władca Pierścieni', tolkien, fantasy).
ksiazka('Mały Książę', saint_exupery, filozoficzna).
ksiazka('Krzyżacy', sienkiewicz, historyczna).
ksiazka('1984', orwell, dystopia).
ksiazka('Dziewczyna z pociągu', hawkins, thriller).
ksiazka('Zbrodnia i kara', dostojewski, psychologiczna).
ksiazka('Lśnienie', king, horror).
ksiazka('Zabić drozda', lee, klasyka).
ksiazka('Wielki Gatsby', fitzgerald, klasyka).
ksiazka('Mistrz i Małgorzata', bulhakow, fantasy).
ksiazka('Przeminęło z wiatrem', mitchell, romans).
ksiazka('Cień wiatru', zafon, thriller).
ksiazka('Shantaram', roberts, przygodowa).
ksiazka('Opowieść podręcznej', atwood, dystopia).
ksiazka('Pamiętnik', sparks, romans).
ksiazka('P.S. Kocham Cię', ahern, romans).

ocena(uzytkownik1, 'W pustyni i w puszczy', 5).
ocena(uzytkownik1, 'Hobbit', 4).
ocena(uzytkownik1, 'Pan Tadeusz', 2).
ocena(uzytkownik1, 'Lalka', 3).
ocena(uzytkownik1, 'Dziady', 5).
ocena(uzytkownik1, 'Zemsta', 2).
ocena(uzytkownik1, 'Koziołek Matołek', 4).
ocena(uzytkownik1, 'Metamorfoza', 3).
ocena(uzytkownik1, 'Harry Potter i Kamień Filozoficzny', 5).
ocena(uzytkownik1, 'Władca Pierścieni', 5).
ocena(uzytkownik1, 'Mały Książę', 1).
ocena(uzytkownik1, 'Krzyżacy', 2).
ocena(uzytkownik1, '1984', 4).
ocena(uzytkownik1, 'Dziewczyna z pociągu', 3).
ocena(uzytkownik1, 'Zbrodnia i kara', 5).
ocena(uzytkownik1, 'Lśnienie', 2).
ocena(uzytkownik1, 'Zabić drozda', 5).
ocena(uzytkownik1, 'Wielki Gatsby', 4).
ocena(uzytkownik1, 'Mistrz i Małgorzata', 3).
ocena(uzytkownik1, 'Przeminęło z wiatrem', 4).
ocena(uzytkownik1, 'Cień wiatru', 5).
ocena(uzytkownik1, 'Shantaram', 2).
ocena(uzytkownik1, 'Opowieść podręcznej', 3).
ocena(uzytkownik1, 'Pamiętnik', 5).
ocena(uzytkownik1, 'P.S. Kocham Cię', 3).
ocena(uzytkownik2, 'W pustyni i w puszczy', 3).
ocena(uzytkownik2, 'Hobbit', 5).
ocena(uzytkownik2, 'Pan Tadeusz', 5).
ocena(uzytkownik2, 'Lalka', 2).
ocena(uzytkownik2, 'Dziady', 4).
ocena(uzytkownik2, 'Zemsta', 5).
ocena(uzytkownik2, 'Koziołek Matołek', 3).
ocena(uzytkownik2, 'Metamorfoza', 4).
ocena(uzytkownik2, 'Harry Potter i Kamień Filozoficzny', 3).
ocena(uzytkownik2, 'Władca Pierścieni', 4).
ocena(uzytkownik2, 'Mały Książę', 5).
ocena(uzytkownik2, 'Krzyżacy', 4).
ocena(uzytkownik2, '1984', 5).
ocena(uzytkownik2, 'Dziewczyna z pociągu', 5).
ocena(uzytkownik2, 'Zbrodnia i kara', 2).
ocena(uzytkownik2, 'Lśnienie', 5).
ocena(uzytkownik2, 'Pamiętnik', 4).
ocena(uzytkownik2, 'P.S. Kocham Cię', 4).
ocena(uzytkownik2, 'Zabić drozda', 4).
ocena(uzytkownik2, 'Wielki Gatsby', 5).
ocena(uzytkownik2, 'Mistrz i Małgorzata', 5).
ocena(uzytkownik2, 'Przeminęło z wiatrem', 3).
ocena(uzytkownik2, 'Cień wiatru', 4).
ocena(uzytkownik2, 'Shantaram', 3).
ocena(uzytkownik2, 'Opowieść podręcznej', 4).
ocena(uzytkownik3, 'W pustyni i w puszczy', 2).
ocena(uzytkownik3, 'Hobbit', 3).
ocena(uzytkownik3, 'Pan Tadeusz', 5).
ocena(uzytkownik3, 'Lalka', 4).
ocena(uzytkownik3, 'Dziady', 3).
ocena(uzytkownik3, 'Zemsta', 4).
ocena(uzytkownik3, 'Koziołek Matołek', 5).
ocena(uzytkownik3, 'Metamorfoza', 2).
ocena(uzytkownik3, 'Harry Potter i Kamień Filozoficzny', 4).
ocena(uzytkownik3, 'Władca Pierścieni', 3).
ocena(uzytkownik3, 'Mały Książę', 4).
ocena(uzytkownik3, 'Krzyżacy', 3).
ocena(uzytkownik3, '1984', 4).
ocena(uzytkownik3, 'Dziewczyna z pociągu', 5).
ocena(uzytkownik3, 'Zbrodnia i kara', 3).
ocena(uzytkownik3, 'Lśnienie', 5).
ocena(uzytkownik3, 'Pamiętnik', 3).
ocena(uzytkownik3, 'P.S. Kocham Cię', 5).
ocena(uzytkownik3, 'Zabić drozda', 3).
ocena(uzytkownik3, 'Wielki Gatsby', 2).
ocena(uzytkownik3, 'Mistrz i Małgorzata', 4).
ocena(uzytkownik3, 'Przeminęło z wiatrem', 5).
ocena(uzytkownik3, 'Cień wiatru', 3).
ocena(uzytkownik3, 'Shantaram', 5).
ocena(uzytkownik3, 'Opowieść podręcznej', 5).

preferencje(uzytkownik1, fantasy).
preferencje(uzytkownik1, przygodowa).
preferencje(uzytkownik1, psychologiczna).
preferencje(uzytkownik1, dramat).
preferencje(uzytkownik2, kryminal).
preferencje(uzytkownik2, klasyka).
preferencje(uzytkownik3, horror).
preferencje(uzytkownik3, romans).


polecane_ksiazki(X, Gatunek) :-
    preferencje(X, Gatunek),  % sprawdza, czy użytkownik X preferuje ten gatunek
    findall(K, (ksiazka(K, _, Gatunek), ocena(X, K, O), O >= 4), P),%K-ksiażka P- polecana książka O-ocena, znajduje ksiazki ktore mają ocenę równą lub większą niż 4 przez użytkowników lubiących ten gatunek
    list_to_set(P, A),  % usuwa duplikaty
    write(A).  % wypisuje listę książek unikajac duplikatow





In [ ]:
?- polecane_ksiazki(uzytkownik1, fantasy). oczekiwane:[Hobbit, Harry Potter i Kamień Filozoficzny, Władca Pierścieni] zwrócone:[Hobbit, Harry Potter i Kamień Filozoficzny, Władca Pierścieni]
?- polecane_ksiazki(uzytkownik2, fantasy). oczekiwane: false zwrocone: false (poniewaz uzytkownik 2 nie preferuje fantasy)
?- polecane_ksiazki(uzytkownik2,  klasyka). zwrócone:[Zabić drozda, Wielki Gatsby]
?- polecane_ksiazki(uzytkownik3,  romans). zwrócone:[Przeminęło z wiatrem, P.S. Kocham Cię]